# FieldTwin API Python Examples

This python notebook runs an example demo of using the FieldTwin API.  
See the API documentation (https://api.fieldtwin.com/).   
   
    
The demo requires an API access token to be entered. Press the run icon in 
the cell below to enter the API access token.

In [ ]:
import getpass

api_token = ''
# User input to enter API token
api_token = getpass.getpass()

# HTTPS GET examples

The following code runs a GET request to read an example project.

In [ ]:
import requests
import json

api_url = 'https://backend.qa.fieldtwin.com/API/v1.10/'

# Field Exporter Integration Testing project
project_id = '-M-HHqMifhz6qskW2goc'
subproject_id = '-M-HI0DPuadaK9GLCWzR'

if api_token:
  # request project data using FieldTwin API
  response = requests.get(
    f'{api_url}{project_id}', 
    headers={'token': api_token}
  )
  project = response.json()

  # check for successful API response
  if (response.status_code == requests.codes.ok):
    print(project['name'])
  else: 
    print('API request failed:', response.status_code)

else:
  print('API access token must be entered')

The following code loads subproject connection data into a Pandas data frame

In [ ]:
import pandas as pd

api_url = 'https://backend.qa.fieldtwin.com/API/v1.10/'

# Field Exporter Integration Testing project
project_id = '-M-HHqMifhz6qskW2goc'
subproject_id = '-M-HI0DPuadaK9GLCWzR'

# the pandas dataframe
df = pd.DataFrame()

if api_token:
  # retrieve the subproject data
  response = requests.get(
    f'{api_url}{project_id}/subProject/{subproject_id}', 
    headers={'token': api_token}
  )

  # check for successful API response
  if (response.status_code != requests.codes.ok):
    print('API request failed:', response.status_code)

  else: 
    # read the subproject data
    subproject = response.json()

    # read the connections data
    connections = subproject['connections']

    items = []
    for connection_id in connections:
      connection = connections[connection_id]
      label = connection.get('params', {}).get('label', 'Un-named Connection')
      item = {
        'Name': label, 
        'Length': connection['length']
      }
      items.append(item)

    # populate the dataframe
    df  = pd.DataFrame(items)
  
else:
  print('API access token must be entered')

Sort the data and display in a table

In [ ]:
from IPython.display import display, HTML

if df.empty:
  print('Data frame is empty')
else:
  df.sort_values(by=['Length'], inplace=True)
  display(HTML(df.to_html()))

Sort and plot the dataframe

In [ ]:
import matplotlib.pyplot as plotter

if df.empty:
  print('Data frame is empty')
else:
  df.sort_values(by=['Name'], inplace=True)
  df.plot(kind = 'scatter', x = 'Length', y = 'Name', figsize=(5, 8))
  plotter.show()

# HTTPS PATCH Example

The following code performs an HTTPS PATCH operation to update a metadata value
for a staged asset.

In [ ]:
api_url = 'https://backend.qa.fieldtwin.com/API/v1.10/'

# Field Exporter Integration Testing project
project_id = '-M-HHqMifhz6qskW2goc'
subproject_id = '-M-HI0DPuadaK9GLCWzR'

# XMT shallow water asset
asset = '-MkCvy0SPaTe65knzJmN'

# Design Life metadata update value
patch_data ='{"metaData": [{"metaDatumId": "-N68U9EVu9djcN8gahs2", "value": 15}]}'

if api_token:
  # invoke an API 'patch' to update the metadata value
  response = requests.patch(
    f'{api_url}{project_id}/subProject/{subproject_id}/stagedAsset/{asset}', 
    data = patch_data,
    headers = {'token': api_token, "Content-Type": "application/json"}
  )
  print(response)

else:
  print('API access token must be entered')



Perform a GET to read back the updated metadata value

In [ ]:
if api_token:
  # invoke a get request for the asset data
  response = requests.get(
    f'{api_url}{project_id}/subProject/{subproject_id}/stagedAsset/{asset}', 
    headers={'token': api_token, "Content-Type": "application/json"}
  )

  # check for successful API response
  if (response.status_code != requests.codes.ok):
    print('API request failed:', response.status_code)

  else: 
    # read the asset metadata
    asset_data = response.json()
    metadata = asset_data['metaData']

    # display the 'Design Life' metadata value
    for item in metadata:
      if item['name'] == 'Design Life':
        value = item["value"]
        unit = item["unit"]["unitDefaultSymbol"]
        print(f'{asset_data["name"]}: {item["name"]}: {value} {unit}')

else:
  print('API access token must be entered')